<a href="https://colab.research.google.com/github/turtle164/AI-ML/blob/main/ran_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
#setup and load the data first
def load_and_prepare_data():
  train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/network_traffic_test.csv')
  test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/network_traffic_training.csv')

  #convert timestamp to datetime
  train_data['timestamp'] = pd.to_datetime(train_data['timestamp'])
  test_data['timestamp'] = pd.to_datetime(test_data['timestamp'])

  #shuffle the databases
  train_data = train_data.sample(frac=1).reset_index(drop=True)
  test_data = test_data.sample(frac=1).reset_index(drop=True)

  return train_data, test_data


In [7]:
train_data, test_data = load_and_prepare_data()

In [9]:
#display first few rows and basic information of dataset
print("Training data shape:",train_data.shape)
print("Test data shape:",test_data.shape)
print("\nTraining data info:")
# Correcting the function call to use the .head() method of the DataFrame
display(train_data.head(20))
display(train_data.info())

Training data shape: (1100, 10)
Test data shape: (11000, 10)

Training data info:


,timestamp,source_ip,dest_ip,protocol,port,bytes,packets,duration,is_attack,attack_type
0,2025-06-17 09:36:27.698628,192.168.172.158,10.0.48.212,UDP,15242,1415.659848,85,0.794371,0,NaN
1,2025-06-18 01:36:09.644932,192.168.194.110,10.0.175.27,UDP,54002,1077.787417,63,6.834831,0,NaN
2,2025-06-17 03:14:03.640438,192.168.58.86,10.0.210.118,TCP,46326,2607.718201,34,29.414025,0,NaN
3,2025-06-17 20:42:02.670758,192.168.203.36,10.0.226.215,ICMP,18663,771.182829,25,12.694634,0,NaN
4,2025-06-17 09:03:12.663418,192.168.104.74,10.0.59.210,TCP,62370,1511.231331,34,51.962200,0,NaN
5,2025-06-17 18:05:51.662666,192.168.239.101,10.0.123.182,TCP,55569,1675.852136,54,138.109273,0,NaN
6,2025-06-17 04:26:16.662809,192.168.65.28,10.0.190.182,TCP,51579,1995.838354,14,33.119020,0,NaN
7,2025-06-17 10:57:58.624427,192.168.248.121,10.0.86.137,TCP,5051,1171.835972,59,12.948526,0,NaN
8,2025-06-17 11:02:56.670108,192.168.64.207,10.0.87.36,UDP,54206,1626.534788,99,56.987198,0,NaN
9,2025-06-17 23:31:15.648928,192.168.225.211,10.0.230.87,TCP,64923,1721.401229,81,15.584381,0,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    1100 non-null   datetime64[ns]
 1   source_ip    1100 non-null   object        
 2   dest_ip      1100 non-null   object        
 3   protocol     1100 non-null   object        
 4   port         1100 non-null   int64         
 5   bytes        1100 non-null   float64       
 6   packets      1100 non-null   int64         
 7   duration     1100 non-null   float64       
 8   is_attack    1100 non-null   int64         
 9   attack_type  100 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 86.1+ KB


None